In [1]:
%matplotlib widget
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1 import make_axes_locatable
import ipywidgets

import numpy as np
import numba
from numba import jit
from scipy import interpolate
from scipy import integrate


@jit(nopython=True)
def points_count(a, b, step):
    return int(np.ceil(np.abs(b - a) / step)) + 1

# Одномерный случай

## Движение одной частицы

Функция $G(x, x_0, t)$ - это плотность вероятности того, что молекула, находившаяся в точке $x_0$ в момент времени $t = 0$, находится в точке $x$ в момент времени $t$. 
Она описывает центростремительное движение молекул на отрезке $[-l, l]$ при следующем линейном распределении интенсивности светового излучения: $I(x) = \frac{I_0}{l} |x|$.

\begin{equation}
    G(x, x_0, t) = 
        \frac{1}{\sigma(t) \sqrt{2 \pi }}
        \exp \left(
            - \frac{1}{2}
            \frac{\left[ x - \mu(x_0, t) \right]^2}{\sigma^2(t)}
        \right),
\end{equation}

\begin{equation}
    \sigma (t) = \sqrt{\frac{k_B T}{k} \left( 1 - \exp \left[ -\frac{2 t}{\tau} \right] \right)},
\end{equation}

\begin{equation}
    \mu(x_0, t) = x_0 \exp \left( -\frac{t}{\tau} \right).
\end{equation}

Единицы измерения приведённых величин в СИ:

* $ \frac{k_B T}{k}$ – $м^2$;
* $ \tau $ - $с$;
* $\sigma(t)$ – $м$;
* $\mu(x_0, t)$ – $м$;
* $G(x, x_0, t)$ – $м^{-1}$.

Функция $G_1(x, x_0, t)$ описывает движение молекулы на отрезке $[0, l]$ при распределении интенсивности $I(x) = I_0 (1 - x/l)$.

\begin{equation}
    G_1(x, x_0, t) = G(x - l, x_0 - l, t).
\end{equation}

Длина отрезка, на котором задано линейное распределение интенсивности, и максимальное значение интенсивности на нём учитывается в параметрах $k$ и $\tau$.

In [54]:
# Reference parameter values [1]
# Phenomenological parameters
a = 43              # nm                           (Initial level of surface)
k_prime = 16        # fN * mW^(-1) cm^(-2)
τ_prime = 3694      # s * mW * cm^(-2) * μm^(-1)

# Other parametsrs
kB = 1.380649e-23         # J * K^(-1) [J = Joule = N * m]   (Boltzmann constant)
T = 298                   # K                                (Absolute temperature)
I0 = 6.25           # mW * cm^(-2)
l = 0.9             # μm

# P = I0 / l       delete this line if everything works

In [55]:
# @jit(nopython=True)
def G(x, x0, t, l, I_max, I_min):
    """
    Units of measurements:
    x, x0, l:  μm
    t:         s
    return:    nm^(-1)
    """
    # TODO: if I == 0, use free diffusion distribution instead (eq. 19 from [2])
    P = (I_max - I_min) / l   # mW * cm^(-2) * μm^(-1)   (Overall intensity gradient)
    k = k_prime * P * 1e-9    # N / m
    τ = τ_prime / P           # s
    x *= 1e-6  
    x0 *= 1e-6
    # now the result will be in nm^(-1)
    var = kB * T / k * (1 - np.exp(-2 * t / τ))  # variance, equals to σ^2
    μ = x0 * np.exp(-t / τ)                      # expected value
    term1 = 1 / np.sqrt(2 * np.pi * var)
    term2 = -0.5 * (x - μ) ** 2 / var
    result = term1 * np.exp(term2)
    # convert arguments back to micrometers
    x *= 1e6
    x0 *= 1e6
    # convert result to nm^(-1): 1 m^(-1) = 1e-9 nm^(-1)
    result *= 1e-9
    return result

# TODO: handle the case when x or/and x0 are outside [0, l] (delta(x0) or free diffusion or smth else)
# @jit(nopython=True)
def G1(x, x0, t, l, I_max, I_min):
    return G(x - l, x0 - l, t, l, I_max, I_min)

def G1_extended(x, x0, t, l, I_max, I_min):
    if x0 < 0 or x0 > l:
        if x == x0:
            if x == x0:
                return 1
            else:
                return 0
    else:
        return G(x - l, x0 - l, t, l, I_max, I_min)
    
# G1 = G1_extended

### График функции $G(x, x_0, t)$ и соответствующего распределения интенсивности

In [4]:
t_start = 10
t_end = 500
t_step = 10
t_pcount = points_count(t_start, t_end, t_step)

x0_start = -l
x0_end = l
x0_step = 0.07
x0_pcount = points_count(t_start, t_end, t_step)

x_pcount = 400
x = np.linspace(-l, l, x_pcount)

plt.close("G plot")
fig00, ax00 = plt.subplots(num="G plot")

def init_G_plot():
    ax00.cla()
    ax00.set_title("График функции $G(x, x_0, t)$")
    ax00.set_xlim(-l, l)
    ax00.set_ylim(0, 0.012)
    ax00.set_xlabel("$x$, мкм")
    ax00.set_ylabel("$G(x, x_0, t)$, нм$^{-1}$")
    
def plot_G_lines(x0):
    t = [10, 20, 30, 40, 60, 100, 200, 300, 400, 500]
    ax00.set_xlim(0, l)
    ax00.set_ylim(0, 0.012)
    ax00.set_xlabel("$x$, мкм")
    ax00.set_ylabel("$G(x, x_0, t)$, нм$^{-1}$")
    for t_val in t:
        ax00.plot(x, G(x, x0, t_val, l, I0, 0), label=f't = {t_val} с')
    ax00.legend()

@ipywidgets.interact(x0=(x0_start, x0_end, x0_step), t=(t_start, t_end, t_step))
def update_G_plot(x0=x0_start, t=t_start):
    init_G_plot()
    ax00.grid(True)
    ax00.axvline(x=x0, color='gray', linestyle='--', label=f'$x_0 = {x0}$ мкм', linewidth=1)
    ax00.plot(x, G(x, x0, t, l, I0, 0), color='C0')
    ax00.legend()
    
# plot_G_lines(0.7)
# fig00.savefig('G(x, x0=0.7, t).png', dpi=400)
    
Ix_G = I0 / l * np.abs(x)

plt.close('intensity for G')
fig01, ax01 = plt.subplots(num='intensity for G')
ax01.grid(True)
ax01.set_title("График соответствующего\nраспределения интенсивности")
ax01.set_xlim(-l, l)
ax01.set_xlabel("$x$, мкм")
ax01.set_ylabel("Интенсивность, мВт/см²")
ax01.plot(x, Ix_G, color='C1');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=-0.9, description='x0', max=0.9, min=-0.9, step=0.07), IntSlider(value…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### График функции $G_1(x, x_0, t)$

In [56]:
t_start = 10
t_end = 500
t_step = 0.001
t_pcount = points_count(t_start, t_end, t_step)

x0_start = 0
x0_end = l
x0_step = 0.07
x0_pcount = points_count(t_start, t_end, t_step)

x_start = -1
x_end = l + 1
x_pcount = 200
x = np.linspace(x_start, x_end, x_pcount)

plt.close(1)
fig1, ax1 = plt.subplots(num=1)


def plot_G_lines(x0):
    t = [10, 20, 30, 40, 60, 100, 200, 300, 400, 500]
    ax1.set_xlim(x_start, x_end)
    ax1.set_ylim(0, 0.012)
    ax1.set_xlabel("$x$, мкм")
    ax1.set_ylabel("$G_1(x, x_0, t)$, нм$^{-1}$")
    for t_val in t:
        ax1.plot(x, G1(x, x0, t_val, l, I0, 0), label=f't = {t_val} с')
    ax1.legend()
    
# plot_G_lines(0.2)
# fig1.savefig('G(x, x0=0.2, t).png', dpi=400)

@ipywidgets.interact(x0=(x0_start, x0_end, x0_step), t=(t_start, t_end, t_step))
def update1(x0=x0_start, t=t_start):
    ax1.cla()
    ax1.grid(True)
    ax1.set_title("Динамика движения одной молекулы")
    ax1.set_xlim(x_start, x_end)
    ax1.set_ylim(0, 0.012)
    ax1.set_xlabel("$x$, мкм")
    ax1.set_ylabel("$G_1(x, x_0, t)$, нм$^{-1}$")
    ax1.axvline(x=x0, color='gray', linestyle='--', label=f'$x_0 = {x0}$ мкм', linewidth=1)
    ax1.axvline(x=0, color='orange', linestyle=':', linewidth=1)
    ax1.axvline(x=l, color='orange', linestyle=':', linewidth=1)
    ax1.plot(x, G1(x, x0, t, l, I0, 0), color='C0')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='x0', max=0.9, step=0.07), FloatSlider(value=10.0, de…

## Динамика одномерного рельефа

In [67]:
def relief1d(x, t, Δx0, l, I_max, I_min):
    x0_pcount = points_count(0, l, Δx0)
    x0 = np.linspace(0, l, x0_pcount)
    xx, x0x0, tt = np.meshgrid(x, x0, t)
    G1_vals = G1(xx, x0x0, tt, l, I_max, I_min)
    # averaging over probabilities for all x0 in [0, l]
    relief_exposed = integrate.simpson(G1_vals, x0, axis=0, even='avg')
    relief_exposed /= l * 1e-3   # why we get perfectly normalized relief function with 1e-3???
    relief_exposed = a * (relief_exposed - 1)  # a is initial level of surface in nm (see parameter values cell)
    return relief_exposed

In [68]:
# TODO: pass meshgrid as parameter so that the function could be compiled with nopython=True
# @jit(forceobj=True)
def relief1d_extended(x, t, Δx0, l, I_max, I_min):
    x0_pcount = points_count(0, l, Δx0)
    x0 = np.linspace(0, l, x0_pcount)
    xx, x0x0, tt = np.meshgrid(x, x0, t)
    G1_vals = G1(xx, x0x0, tt, l, I_max, I_min)
    # averaging over probabilities for all x0 in [0, l]
    relief_exposed = integrate.simpson(G1_vals, x0, axis=0, even='avg')
    relief_exposed /= l * 1e-3   # why we get perfectly normalized relief function with 1e-3???
    relief_outside = np.where((0 <= x) & (x <= l), 0, 1)
    relief = relief_exposed + relief_outside[:, None]
    relief = a * (relief - 1)  # a is initial level of surface in nm (see parameter values cell)
    return relief

### График функции $G1(x,x0,t)$ при $t = const$

In [69]:
%matplotlib widget

x_start = -1
x_end = l + 1
x_pcount = 200
x = np.linspace(x_start, x_end, x_pcount)
x0 = np.linspace(0, l, x_pcount)
t = 250

xx, x0x0 = np.meshgrid(x, x0)
G1_vals = G1(xx, x0x0, t, l, I0, 0)

fig20 = plt.figure(num=20)
ax20 = fig20.add_subplot(1, 1, 1, projection='3d')
ax20.plot_surface(xx, x0x0, G1_vals)
ax20.plot(np.linspace(x_start, 0), np.linspace(x_start, 0), zs=0, zdir='z', color='green')
ax20.plot(np.linspace(l, x_end), np.linspace(l, x_end), zs=0, zdir='z', color='green')
ax20.set_xlabel('$x$')
ax20.set_ylabel('$x_0$')
ax20.set_zlabel('$G_1$')
ax20.set_ylim(x_start, x_end)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-1.0, 1.9)

In [73]:
plt.close(2)
fig2, ax2 = plt.subplots(num=2)

plt.close(21)
fig21, ax21 = plt.subplots(num=21)

t_start = 1
t_end = 500
t_step = 1
t_pcount = points_count(t_start, t_end, t_step)
t = np.linspace(t_start, t_end, t_pcount)
Δx0 = 0.01

x_start = -1
x_end = l + 1
x_pcount = 200
x = np.linspace(x_start, x_end, x_pcount)

x0_start = x_start
x0_end = x_end
x0 = np.linspace(x0_start, x0_end, x0_pcount)

relief = relief1d(x, t, Δx0, l, I0, 0)
relief_extended = relief1d_extended(x, t, Δx0, l, I0, 0)

def plot_relief_lines():
    t = [50, 100, 200, 300, 500]
    ax2.set_xlabel("$x$, мкм")
    ax2.set_ylabel("Высота, нм")
    ax2.set_xlim(x_start, x_end)
    ax2.set_ylim(np.min(relief), np.max(relief))
    for t_val in t:
        ax2.plot(x, relief1d(x, t_val, Δx0, l, I0, 0), label=f't = {t_val} с')
    ax2.legend()
    
# plot_relief_lines()
# fig2.savefig('reliefs.png', dpi=400)

@ipywidgets.interact(t_idx=(0, t.size - 1, 1))
def update2(t_idx=0):
    ax2.cla()
    ax2.grid(True)
    ax2.set_xlabel("$x$, мкм")
    ax2.set_ylabel("Высота, нм")
    ax2.set_title("Динамика одномерного рельефа")
    ax2.set_xlim(x_start, x_end)
    ax2.set_ylim(np.min(relief_extended), np.max(relief_extended))
    ax2.axvline(x=0, color='orange', linestyle=':', linewidth=1)
    ax2.axvline(x=l, color='orange', linestyle=':', linewidth=1)
    ax2.plot(x, relief[:, t_idx], color='C2', label=f't = {t[t_idx]} s')
    ax2.legend()

@ipywidgets.interact(t_idx=(0, t.size - 1, 1))
def update2_extended(t_idx=0):
    ax21.cla()
    ax21.grid(True)
    ax21.set_xlabel("$x$, мкм")
    ax21.set_ylabel("Высота, нм")
    ax21.set_title("Динамика одномерного рельефа\n(расширенный вариант)")
    ax21.set_xlim(x_start, x_end)
    ax21.set_ylim(np.min(relief_extended), np.max(relief_extended))
    ax21.axvline(x=0, color='orange', linestyle=':', linewidth=1)
    ax21.axvline(x=l, color='orange', linestyle=':', linewidth=1)
    ax21.plot(x, relief_extended[:, t_idx], color='C2', label=f't = {t[t_idx]} s')
    ax21.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='t_idx', max=499), Output()), _dom_classes=('widget-inter…

interactive(children=(IntSlider(value=0, description='t_idx', max=499), Output()), _dom_classes=('widget-inter…

# Двумерный случай

In [19]:
# @jit(forceobj=True)
def linear_dynamics2d(x1, x2, y1, y2, I, ξ_pcount, η_pcount, t, Δx0, ax=None):
    """
    I - function of a linear intensity in the rectangular region (scipy.interpolate.interp2d object)
    """
    # Calculate parameters
    lx = x2 - x1
    ly = y2 - y1
    grad_I = ((I(x2, y1) - I(x1, y1)).item() / lx,
              (I(x1, y2) - I(x1, y1)).item() / ly)
    θ = np.arctan2(-grad_I[1], -grad_I[0])
    cosθ = np.cos(θ)
    sinθ = np.sin(θ)
    lξ = lx * np.abs(cosθ) + ly * np.abs(sinθ)
    lη = lx * np.abs(sinθ) + ly * np.abs(cosθ)
    ξ = np.linspace(0, lξ, ξ_pcount)
    η = np.linspace(0, lη, η_pcount)
    
    # Transform ξη to xy coordinates (rotate counterclockwise and shift)
    ξξ, ηη = np.meshgrid(ξ, η)
    if 0 <= θ <= np.pi / 2:
        Δx = x1 + lx * sinθ * sinθ
        Δy = y1 - lx * sinθ * cosθ
    elif np.pi / 2 <= θ <= np.pi:
        Δx = x2 - ly * cosθ * sinθ
        Δy = y1 + ly * cosθ * cosθ
    elif -np.pi <= θ <= -np.pi / 2:
        Δx = x2 - lx * sinθ * sinθ
        Δy = y2 + lx * sinθ * cosθ
    elif -np.pi / 2 <= θ <= 0:
        Δx = x1 + ly * cosθ * sinθ
        Δy = y2 - ly * cosθ * cosθ
    xx = cosθ * ξξ - sinθ * ηη + Δx
    yy = sinθ * ξξ + cosθ * ηη + Δy
    # Filter out points that are outside of the rectangular region.
    # The grid becomes irregular (contains np.nan's).
    x_mask = np.logical_and(xx >= x1, xx <= x2)
    y_mask = np.logical_and(yy >= y1, yy <= y2)
    region_mask = x_mask & y_mask
    xx_region = np.where(region_mask, xx, np.nan)
    yy_region = np.where(region_mask, yy, np.nan)
    # remove rows that contain only nan's
    xx_region = xx_region[~np.isnan(xx_region).all(axis=1)]
    yy_region = yy_region[~np.isnan(yy_region).all(axis=1)]
    # Transform points back to ξη coordinates
    ξξ_region =  cosθ * (xx_region - Δx) + sinθ * (yy_region - Δy)
    ηη_region = -sinθ * (xx_region - Δx) + cosθ * (yy_region - Δy)
    
    # 1D arrays defining the data point coordinates
    ξ_relief = np.array([])
    η_relief = np.array([])
    relief_region = np.array([])
    # Compute relief for every line
    # TODO: eliminate the loop!
    for i in range(ξξ_region.shape[0]):
        ξ_reg_line = ξξ_region[i][~np.isnan(ξξ_region[i])]
        η_reg_line = ηη_region[i][~np.isnan(ηη_region[i])]
        x_reg_line = xx_region[i][~np.isnan(xx_region[i])]
        y_reg_line = yy_region[i][~np.isnan(yy_region[i])]
        I_max = I(x_reg_line[0], y_reg_line[0]).item()
        I_min = I(x_reg_line[-1], y_reg_line[-1]).item()
        relief_reg_line = relief1d(ξ_reg_line, t, Δx0, l, I_max, I_min).squeeze()
        ξ_relief = np.concatenate((ξ_relief, ξ_reg_line))
        η_relief = np.concatenate((η_relief, η_reg_line))
        relief_region = np.concatenate((relief_region, relief_reg_line))
    # Finally, transform points to xy coordinates
    x_relief = cosθ * ξ_relief - sinθ * η_relief + Δx
    y_relief = sinθ * ξ_relief + cosθ * η_relief + Δy
        
    # visualize
    if ax is not None:
        ax.add_patch(Rectangle((x1, y1), lx, ly, fill=False, edgecolor='black'))
        # grad f
        x_mid = x1 + lx / 2
        y_mid = y1 + ly / 2
        ax.arrow(x_mid, y_mid, grad_I[0] * 0.1, grad_I[1] * 0.1, shape='full', width=0.01, head_width=0.05, color='red', label='grad $\widetilde{I}(x,y)$')
        # ξ and η axes
        ax.arrow(xx[0][0], yy[0][0], xx[0][-1] - xx[0][0], yy[0][-1] - yy[0][0], shape='full', width=0.01, head_width=0.05, color='orange')
        ax.annotate('ξ', (xx[0][-1], yy[0][-1]), fontsize='large')
        ax.arrow(xx[0][0], yy[0][0], xx[-1][0] - xx[0][0], yy[-1][0] - yy[0][0], shape='full', width=0.01, head_width=0.05, color='orange')
        ax.annotate('η', (xx[-1][0], yy[-1][0]), fontsize='large')
        # grid points
        ax.scatter(xx, yy, marker='.')
        ax.scatter(xx_region, yy_region, marker='+')
        ax.scatter(x_relief, y_relief)
    
    return x_relief, y_relief, relief_region

# @jit()
def dynamics2d(x_reg_points, y_reg_points, I, ξ_pcount, η_pcount, t, Δx0, ax=None):
    x_relief = np.array([])
    y_relief = np.array([])
    relief = np.array([])
    for i in range(x_reg_points.size - 1):
        for j in range(y_reg_points.size - 1):
            x1 = x_reg_points[i]
            x2 = x_reg_points[i + 1]
            y1 = y_reg_points[j]
            y2 = y_reg_points[j + 1]
            x_reg_relief, y_reg_relief, reg_relief = \
                linear_dynamics2d(x1, x2, y1, y2, I, ξ_pcount, η_pcount, t, Δx0, ax=ax)
            x_relief = np.concatenate((x_relief, x_reg_relief))
            y_relief = np.concatenate((y_relief, y_reg_relief))
            relief = np.concatenate((relief, reg_relief))
    return x_relief, y_relief, relief

## В следующей клетке моделируется динамика рельефа для одной области с линейной интенсивностью.

In [20]:
plt.close(3)
fig3, ax3 = plt.subplots(num=3)

plt.close(4)
fig4, ax4 = plt.subplots(num=4)
divider4 = make_axes_locatable(ax4)
cax4 = divider4.append_axes('right', size='5%', pad=0.2)

@ipywidgets.interact(a=(-5, 5, 0.2), b=(-5, 5, 0.2))
def update3(a=-3, b=-1):
    ax3.cla()
    ax3.set_xlim(-1, 1)
    ax3.set_ylim(-1, 1)
    ax3.set_xlabel("$x$, мкм")
    ax3.set_ylabel("$y$, мкм")

    def f(x, y):
        return a * x + b * y + 1

    x_start = -0.5
    x_end = 0.5
    x_pcount = 40
    x_reg_pcount = 2
    y_start = -0.4
    y_end = 0.4
    y_pcount = 40
    y_reg_pcount = 2

    x = np.linspace(x_start, x_end, x_pcount)
    y = np.linspace(y_start, y_end, y_pcount)
    xx, yy = np.meshgrid(x, y)
    f_vals = f(xx, yy)

    # Divide on regions with linear intensity
    x_reg_points = np.linspace(x_start, x_end, x_reg_pcount)
    y_reg_points = np.linspace(y_start, y_end, y_reg_pcount)
    xx_reg_points, yy_reg_points = np.meshgrid(x_reg_points, y_reg_points)
    f_reg_vals = f(xx_reg_points, yy_reg_points)

    f_lin = interpolate.interp2d(x_reg_points, y_reg_points, f_reg_vals, bounds_error=True)
    
    t = 500
    Δx0 = 0.1
    ξ_pcount = 25
    η_pcount = 25
    
    x_relief, y_relief, relief = \
        dynamics2d(x_reg_points, y_reg_points, f, ξ_pcount, η_pcount, t, Δx0, ax=ax3)
    
    x1 = x_reg_points[0]
    x2 = x_reg_points[1]
    y1 = y_reg_points[0]
    y2 = y_reg_points[1]
    ax3.scatter(x1, y1, c='green')
    ax3.annotate('$(x_{m}, y_{n})$', (x1, y1))
    ax3.scatter(x2, y2, c='green')
    ax3.annotate('$(x_{m+1}, y_{n+1})$', (x2, y2))
    ax3.legend()
    # fig3.savefig('approx region', dpi=400)

    # interpolate data so that it can be plotted
    points = np.stack((x_relief, y_relief)).T
    relief_grid = interpolate.griddata(points, relief, (xx, yy), method='linear')
    
    # plot 2d relief as heat map
    ax4.cla()
    cax4.cla()
    ax4.set_title('Динамика двумерного рельефа')
    ax4.set_xlabel('$x$, мкм')
    ax4.set_ylabel('$y$, мкм')
    plot4 = ax4.imshow(relief_grid, extent=(x_start, x_end, y_start, y_end), origin='lower')
    cbar4 = fig4.colorbar(plot4, cax=cax4)
    cbar4.set_label('Высота, нм')
    # fig4.savefig('approx region relief', dpi=400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=-3.0, description='a', max=5.0, min=-5.0, step=0.2), FloatSlider(value…

## Динамика двумерного рельефа для произвольного распределения интенсивности

In [10]:
def I_norm(x, y):
    # I_norm(0, 0) = 1
    # I_norm(l, 0) = I_norm(0, l) = 0
    return -l**(-2) * (x**2 + y**2) + 1

def I(x, y):
    return I0 * I_norm(x, y)

x_start = -l
x_end = l
x_pcount = 300
x_reg_pcount = 11
y_start = -l
y_end = l
y_pcount = 300
y_reg_pcount = 11

x = np.linspace(x_start, x_end, x_pcount)
y = np.linspace(y_start, y_end, y_pcount)
xx, yy = np.meshgrid(x, y)
I_vals = I(xx, yy)

# Divide on regions with linear intensity
x_reg_points = np.linspace(x_start, x_end, x_reg_pcount)
y_reg_points = np.linspace(y_start, y_end, y_reg_pcount)
xx_reg_points, yy_reg_points = np.meshgrid(x_reg_points, y_reg_points)
I_reg_vals = I(xx_reg_points, yy_reg_points)

I_lin = interpolate.interp2d(x_reg_points, y_reg_points, I_reg_vals, kind='linear', bounds_error=True)
I_lin_vals = I_lin(x, y)

t_start = 1
t_end = 500
t_step = 1
t_pcount = points_count(t_start, t_end, t_step)
# t = np.linspace(t_start, t_end, t_pcount)
t = 100
Δx0 = 0.1
ξ_pcount = 50
η_pcount = 50

x_relief, y_relief, relief = \
    dynamics2d(x_reg_points, y_reg_points, I, ξ_pcount, η_pcount, t, Δx0, ax=None)

# interpolate data so that it can be plotted
points = np.stack((x_relief, y_relief)).T
relief_grid = interpolate.griddata(points, relief, (xx, yy), method='cubic')

# plot 2d relief
plt.close(5)
fig5, ax5 = plt.subplots(num=5)
divider5 = make_axes_locatable(ax5)
cax5 = divider5.append_axes('right', size='5%', pad=0.2)
ax5.set_title('Динамика двумерного рельефа')
ax5.set_xlabel('$x$, мкм')
ax5.set_ylabel('$y$, мкм')
plot5 = ax5.imshow(relief_grid, extent=(x_start, x_end, y_start, y_end), origin='lower')
cbar5 = fig5.colorbar(plot5, cax=cax5)
cbar5.set_label('Высота, нм')

# fig5.savefig('2d dynamics', dpi=400)

# plot intensity
plt.close(6)
fig6, ax6 = plt.subplots(num=6)
divider6 = make_axes_locatable(ax6)
cax6 = divider6.append_axes('right', size='5%', pad=0.2)
ax6.set_title('Интенсивность излучения')
ax6.set_xlabel('$x$, мкм')
ax6.set_ylabel('$y$, мкм')
plot6 = ax6.imshow(I_lin_vals, extent=(x_start, x_end, y_start, y_end), origin='lower', cmap='copper')
cbar6 = fig6.colorbar(plot6, cax=cax6)
cbar6.set_label('Интенсивность, мВт/см²')
               
# fig6.savefig('2d dynamics intensity', dpi=400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Bibliography

1) [Nesterenko, Dmitry V., et al. "Simulation of photochemically induced motion of matter in gradient light fields." Journal of Applied Physics 127.24 (2020): 243106.](https://doi.org/10.1063/5.0011388)

2) [Sekkat, Zouheir. "Optical tweezing by photomigration." Applied Optics 55.2 (2016): 259-268.](https://doi.org/10.1364/AO.55.000259)